In [1]:
import matplotlib.pyplot as plt
import altair as alt
import streamlit as st
from streamlit_folium import st_folium
import geopandas as gpd
import folium
import pandas as pd
from shapely.geometry import Point
# import os

In [ ]:
nodes = pd.read_csv('input/nodesGEOM.txt', sep='\t', header=0)
def join_geom(file, geom = nodes, **kwargs):
    data = pd.read_csv(file, sep = '\t', header = None, names = ['gridID', 'value1', 'value2', 'value3'])
    for col in data.columns:
        if data[col].isnull().values.all():
            data.drop(labels=col, axis=1, inplace=True)
    data = geom.merge(data, on = 'gridID', how = 'right')
    return data
data3col = join_geom('input/q.txt')
data3col.head()

In [7]:
file = 'input/treat_freq.txt'
# geom = gpd.read_file('input/gridsGEO.shp', driver='ESRI Shapefile')
data = pd.read_csv(file, sep = '\t', header = None, names = ['gridID', 'value'])
# data = geom.merge(data, on = 'gridID', how = 'right')
data.head(3)

,gridID,value
0,920,0.0714
1,923,0.1000
2,937,0.1000


In [ ]:
center = [data3col['Lat'].mean(), data3col['Long'].mean()]
center

In [ ]:
dataGJ = data.__geo_interface__
dataGJ

In [ ]:
import leafmap.foliumap as leafmap

In [ ]:
m = leafmap.Map(location=center, zoom_start=6)
m.add_geojson(dataGJ, layer_name='grids')
# m.add_points_from_xy(
#     data3col,
#     x='Lat',
#     y='Long',
#     color_coliumn='value1',
#     add_legend=True,
# )
# m.to_streamlit(height=700)
m

In [ ]:


m = data.explore(
    m=m,
    column='value2',
    tooltip={"gridID", 'value1', 'value2'},
    camp='summer',
    height=400,
    width=400
)
m

In [ ]:
#Load your x-y data - path 
xcoord = pd.read_csv('input/x_coord.txt', sep = '\t', header = None, names = ['gridID', 'x'])
ycoord = pd.read_csv('input/y_coord.txt', sep = '\t', header = None, names = ['gridID', 'y'])
xcoord = xcoord.merge(ycoord, on = 'gridID', how = 'left')
SRID = 4269
geom = [Point(xy) for xy in zip(xcoord['x'], xcoord['y'])]
geom = gpd.GeoDataFrame(xcoord, crs = SRID, geometry = geom )

In [ ]:
geom = pd.read_csv('input/nodesGEOM.txt', sep='\t', header=0)
geom.head()

In [ ]:
pts = [[x,y] for x,y in zip(geom['Long'], geom['Lat'])]
pts

In [ ]:
# joining geometry with NodeID - value format input 
def join_geom(file, geom = geom, **kwargs):
    data = pd.read_csv(file, sep = '\t', header = None, names = ['gridID', 'period', 'value'])
    data = geom.merge(data, on = 'gridID', how = 'right')
    # data['period'] = 'P' + data['period'].astype(str)
    return data#, len(set(data['period']))

In [3]:
grids = gpd.read_file('input/gridsGEO.shp', driver='ESRI Shapefile')
def join_geom(file, geom = grids, **kwargs):
    data = pd.read_csv(file, sep = '\t', header = None, names = ['gridID', 'value'])
    data = geom.merge(data, on = 'gridID', how = 'right')
    return data
data = join_geom('input/treat_freq.txt')
data.head()

,gridID,geometry,value
0,920,"POLYGON ((-114.82544 54.07963, -114.80164 54.0...",0.0714
1,923,"POLYGON ((-114.60337 54.12143, -114.57981 54.0...",0.1000
2,937,"POLYGON ((-113.56059 54.31049, -113.53813 54.2...",0.1000
3,1028,"POLYGON ((-114.25539 54.23371, -114.23217 54.1...",0.1111
4,1037,"POLYGON ((-113.58310 54.35426, -113.56059 54.3...",0.1111


In [ ]:
data.explore(column='value', scheme='Quantiles', cmap='Blues')

In [ ]:

def first_time(data):
    data = data.sort_values('period')
    data.drop_duplicates(subset='gridID', keep='first', inplace=True)
    return data

In [6]:
data = pd.read_csv('input/q.txt', sep = '\t', header = None, names = ['gridID', 'period', 'value'])
data.head()

,gridID,period,value
0,920,9,1
1,923,14,1
2,937,13,1
3,1028,14,1
4,1037,14,1


In [ ]:
data.at[2,'value'] = 32
check = data['value'].isnull().values.all()
check

In [ ]:
def join_geom(file, geom = grids, **kwargs):
    data = pd.read_csv(file, sep = '\t', header = None, names = ['gridID', 'value'])
    data = geom.merge(data, on = 'gridID', how = 'right')
    return data

In [ ]:
# rawdata= pd.read_csv('input/q.txt', sep = '\t', header = None, names = ['gridID', 'value'])
data = join_geom('input/q.txt')
data.head()

In [ ]:
check = data['geometry'].isnull().values.all()
check

In [ ]:
data= first_time(rawdata)
data

In [ ]:
xcoord.head()

In [ ]:
df = first_time(join_geom('input/q.txt', geom=xcoord))

In [ ]:
import altair as alt
# alt.data_transformers.disable_max_rows()

In [ ]:
alt.Chart(data).mark_geoshape().encode(
    color='value:Q'
)

In [ ]:
a = alt.Chart(xcoord).mark_point(filled=False, size=15).encode(
            alt.X('x').scale(zero=False).axis(labels=False), 
            alt.Y('y').scale(zero=False).axis(labels=False),
            color=alt.value("#808080"),
            )#.configure_axis(grid=False)
a

In [ ]:
c = alt.Chart(df).mark_point(filled=True, size=20).encode(
            alt.X('x').scale(zero=False).axis(labels=False), 
            alt.Y('y').scale(zero=False).axis(labels=False),
            color="period:N",
            shape="period:N",
            ).configure_axis(grid=False)
c

In [ ]:
import geojson
with open('input/alberta.geojson') as f:
    alberta = geojson.load(f)
type(alberta)

In [ ]:
b = alt.Chart(alberta).mark_geoshape(
    fill='lightgray',
    stroke='white'
).properties(
    width=500,
    height=300
).project('albersUsa')
b

In [ ]:
from vega_datasets import data
airports = data.airports()
airports.head()

In [ ]:
a = alt.Chart(airports).mark_circle().encode(
    longitude='longitude:Q',
    latitude='latitude:Q',
    size=alt.value(10),
    tooltip='name'
).project(
    "albersUsa"
).properties(
    width=500,
    height=400
)
a

In [ ]:
states = alt.topo_feature(data.us_10m.url, feature='states')

b = alt.Chart(states).mark_geoshape(
    fill='lightgray',
    stroke='white'
).project('albersUsa').properties(
    width=500,
    height=400
)
states

In [ ]:
b + a

In [ ]:
abshp = gpd.read_file(r'H:\DATA\MPB_Ning\shp\alberta_prj.shp', driver='ESRI Shapefile')
shape = abshp["geometry"].apply(lambda p: list(p.exterior.coords)).explode().apply(pd.Series).rename(columns=({0:"x", 1:"y"}))
shape = shape.reset_index(drop=True)
# shape = shape.reset_index()

In [ ]:
shape.to_csv('input/alberta_xy.csv', index=False)

In [ ]:
background = alt.Chart(shape).mark_line().encode(
    alt.X('x').scale(zero=False).axis(labels=False), 
    alt.Y('y').scale(zero=False).axis(labels=False),
    order='index')#.configure_axis(grid=False)
background

In [ ]:
c = alt.Chart(df).mark_point(filled=True, size=20).encode(
            alt.X('x').scale(zero=False).axis(labels=False), 
            alt.Y('y').scale(zero=False).axis(labels=False),
            color="period:N",
            shape="period:N",
            )#.configure_axis(grid=False)
background + c

In [ ]:
from geojson import dump
with open('input/alberta.geojson', 'w') as f:
    dump(abgeojson, f)

In [ ]:
abgeojson.